The task is to find the shortest overall route between many destinations: saleswoman visits several stores in succession and returns to the starting point in the overall shortest distance at the end.

In [1]:
                  import pandas as pd
from random import randint, shuffle, choices, random
from heapq import nlargest
from copy import deepcopy

In [2]:
df = pd.read_excel("DistanceStores22.xlsx")
df['DistanceKM'] = ((df['Distance(m)']/1000).astype(int)).astype(str)
keys = zip(df['StartLocation'],df['Destination'])
values = df['DistanceKM'].astype(int)
source_distances = dict(zip(keys, values))
distances = {}

#Background:

Imagine a sales person for a golden burger chain in Moscow, once upon a time. Sales person has to travel four stores A, B, C, and D. Since time is money, it is only natural that you want to find the overall shortest traveling route.

First of all we need the distances between all of the four stores:

In [9]:
locations = sorted(list(set(df['StartLocation']).union(set(df['Destination']))))
for loc1 in locations:
    distances[loc1] = {}
    for loc2 in locations:
        if loc1 == loc2:
            distances[loc1][loc2] = 0
        elif (loc1, loc2) in source_distances:
            distances[loc1][loc2] = source_distances[(loc1, loc2)]
        elif (loc2, loc1) in source_distances:
            distances[loc1][loc2] = source_distances[(loc2, loc1)]
        else:
            distances[loc1][loc2] = float('inf')

four_stores = locations[:4]

for i in range(len(four_stores)):
    for j in range(i + 1, len(four_stores)):
        store1 = four_stores[i]
        store2 = four_stores[j]
        distance = distances[store1][store2]
        print(f"Distance between {store1} and {store2}: {distance} km")

Distance between StoreA and StoreB: 15 km
Distance between StoreA and StoreC: 8 km
Distance between StoreA and StoreD: 10 km
Distance between StoreB and StoreC: 10 km
Distance between StoreB and StoreD: 10 km
Distance between StoreC and StoreD: 2 km


As we can see the distance of all stores -in both directions to each other- are known. We are considering both directions, because distance by car can slightly vary.

To combine all stores in one overall shortest route is called a nondeterministic polynomial problem. This means that a found shortest distance can be verified at most in polynomial time. Colloquially speaking, it can take a long time to find the optimal route with a high degree of certainty. Since we don’t have this time (and even if we did, we can’t prove whether we really found the shortest total distance or not) we will approach the best possible solution through a special method called genetic algorithm.

In [10]:
for places in source_distances:
    distances[places] = source_distances[places]


#Solution:

Genetic algorithm repeatedly modifies a population of chromosomes (individual solutions). A chromosome contains one of the possible sequences in which the stores could be traveled.

The idea behind chromosomes is based on a natural selection process that mimics biological evolution. Since it only mimics, but certainly does not even come close to the mirror of nature, I prefer to call the genetic approach “advanced blind guess” and the chromosomes “potential solution”. The interesting aspect of the genetic approach is that at each step, the genetic algorithm randomly selects individuals from the current population and uses them as parents to produce the children for the next generation. So by using genetic algorithm, particularly promising routes (sequence order of stores) are selected from an initial population of possible, randomly generated solutions. After a certain number of cycles (mutation) the population “evolves” toward an optimal solution.

The implementation of the genetic algorithm consists of the class PotentialSolution, which represents the chromosomes as one possible single route solution.

In [4]:
class PotentialSolution:
    def randomInstance(cls):
        return PotentialSolution()

    def fitness(self):
        return 1

    def mutate(self):
        return self

    def crossover(self, other):
        return [self, other]

randomInstance() creates an instance with random places for the first generation. It is also capable of mutating, i.e. changing a randomly selected route, and performing a crossover with another chromosome to swap places with it randomly.

Since the distance is to be minimized, the fitness function calculates the reciprocal of the respective total distance.

During mutation, the positions of two places are randomly swapped,

and in a crossover, a store swaps its position with the position of that store in another chromosome.

The class method randomInstance() first creates a copy of the places list, since otherwise the same list would always be passed around.

The copy is randomized using the random shuffle() function and used to create a new instance, which is then returned.


In [5]:
class AdvancedGuess:

    def __init__(self, population, expected, max_generations,
            crossover_chance, mutation_chance):
        self.population = population
        self.expected = expected
        self.max_generations = max_generations
        self.crossover_chance = crossover_chance
        self.mutation_chance = mutation_chance

    def chooseParents(self):
        pool = choices(self.population, k = len(self.population) // 4)
        return nlargest(2, pool, key = lambda potentialSolution: potentialSolution.fitness())

    def propagate(self):
        newPopulation = []
        while len(newPopulation) < len(self.population):
            parents = self.chooseParents()
            if random() < self.crossover_chance:
                [child1, child2] = parents[0].crossover(parents[1])
                newPopulation.append(child1)
                newPopulation.append(child2)
            else:
                newPopulation.append(parents[0])
                newPopulation.append(parents[1])
        if len(newPopulation) > len(self.population):
            newPopulation.pop()
        for potentialSolution in newPopulation:
            if random() < self.mutation_chance:
                potentialSolution.mutate()
        self.population = newPopulation

    def find(self):
        optimal = deepcopy(
            max(self.population, key = lambda potentialSolution: potentialSolution.fitness())
        )
        for i in range(0, self.max_generations):
            if optimal.fitness() >= self.expected:
                return optimal
            self.propagate()
            current_best = deepcopy(
                max(self.population, key = lambda potentialSolution: potentialSolution.fitness())
            )
            if current_best.fitness() > optimal.fitness():
                optimal = current_best
            print(i, optimal)
        return optimal

In [6]:
class distanceShuffling(PotentialSolution):
    locations = df['StartLocation'].unique().tolist() #["StoreA", "StoreB", "StoreC","StoreD"]
    distance =distances # {('StoreA', 'StoreB'): 15, ('StoreB', 'StoreA'): 15, ('StoreC', 'StoreB'): 10,
    def __init__(self, places):
        self.places = places

    def getDistance(self):
        sum = 0
        for index, store in enumerate(self.places):
            if index < len(self.places) - 1:
                nextStore = self.places[index + 1]
            else:
                nextStore = self.places[0]
            sum += self.distance[(store, nextStore)] # C zu D 2 km, A zu D 10km
        return sum # total sum of distance per first combination:D,B,C,A -->38km; danach nächste Schleife, wieder beginnend Index0

    def fitness(self):
        return 1 / self.getDistance()

    @classmethod
    def randomCoordinates(cls):
        PlacesCopy = cls.locations[:]
        shuffle(PlacesCopy)
        return distanceShuffling(PlacesCopy)

    def mutate(self):
        rand_index_1 = randint(0, len(self.places) - 1)
        rand_index_2 = randint(0, len(self.places) - 1)
        if rand_index_1 != rand_index_2:
            self.places[rand_index_1], self.places[rand_index_2] = (
                self.places[rand_index_2], self.places[rand_index_1]
            )

    def crossover(self, other):
        child1 = deepcopy(self)
        child2 = deepcopy(other)
        rand_index = randint(0, len(child1.places) - 1)
        store = child1.places[rand_index]
        other_index = child2.places.index(store)
        if rand_index != other_index:
            child1.places[rand_index], child1.places[other_index] = (
                child1.places[other_index], child1.places[rand_index]
            )
            child2.places[rand_index], child2.places[other_index] = (
                child2.places[other_index], child2.places[rand_index]
            )
        return [child1, child2]

    def __str__(self):
        result = " - ".join(self.places)
        result += " - " + self.places[0]
        result += ": " + str(self.getDistance())
        return result

The copy is randomized using the random shuffle() function and used to create a new instance, which is then returned.



In mutate() and crossover(), the randint() function is used to generate a random integer between the two specified numbers. The method crossover() obtains complete copies of the parent chromosomes with copy.deepcopy() before swapping information between the children. This is because the parents could otherwise be reselected in the current reproduction loop.

The main program creates an initial first population of 20 random chromosomes. The sum of all distances of each chromosomes are calculated. As soon as its fitness exceeds a specified threshold (a very tiny number), the generation sequence ends prematurely and the optimal solution is returned. Otherwise the best one is found after a predetermined number of generations. The algorithm will stop after the maximum number of 20 generations is reached. Crossover_chance is a probability value between 0 and 1 that indicates how often on average a crossover should occur instead of a direct takeover of the parent generation. Mutation_chance specifies how often chromosomes of a new generation should mutate on average. That value is also between 0 and 1.

The main process takes place in the find() method.

This first determines the fittest chromosome of the initial generation before starting the loop over at most max_generations generations. If the previous best value already reaches the threshold expected, it is returned immediately. Both the overall fittest chromosome and the fittest of each round are duplicated.

In the standard case, the reproductive cycle for the next generation occurs next. The method propagate() is responsible for this. Afterwards the best individual of the new generation is determined, compared with the previous best value and taken over, if a further improvement has occurred. If the selected maximum number of generations was run through and no optimal result came out, finally the best result found so far is returned. The propagate() reproduction method also loops, selecting or creating individuals for the next generation until there are enough. For this purpose, two individuals are first selected in each loop pass.

A random value compared with the crossover probability next decides whether the original individuals from the previous generation will be carried over or whether a new generation will be created by their parents.

Before the new generation finally replaces the old one, another loop over all new individuals determines whether they should mutate or not. This is also done by comparing a random value with the desired probability.

In [7]:
if __name__ == '__main__':
    population = []
    for i in range(0, 20):
        population.append(distanceShuffling.randomCoordinates())
    advancedGuessing = AdvancedGuess(population, 0.0000001, 20, 0.7, 0.5)
    optimal = advancedGuessing.find()
    print("Shortest distance found:", optimal)

Shortest distance found: StoreD - StoreC - StoreA - StoreB - StoreD: 35


As soon as we run all of the code, we get a solution within just a few seconds

#Explanation

The code aims to solve the Traveling Salesperson Problem (TSP), which involves finding the shortest possible route that visits a set of locations and returns to the starting point. In this case, the locations are stores that a salesperson needs to visit.

The Approach Since TSP is a computationally complex problem, the code utilizes a genetic algorithm to find a near-optimal solution. Genetic algorithms are inspired by natural selection and evolution. They work by iteratively improving a population of potential solutions (called chromosomes) over multiple generations.

Here's a breakdown of how the code works:

**Initialization:**

A set of potential routes (chromosomes) is randomly generated, representing the initial population.
Each route is a permutation of the stores to be visited.

**Evaluation:**

Each route's "fitness" is calculated, which is inversely proportional to the total distance of the route. Shorter routes have higher fitness.

**Selection:**

Routes with higher fitness (shorter distances) are more likely to be selected as "parents" for the next generation. This selection process is based on the principles of natural selection.

**Reproduction:**

        New routes (offspring) are generated by combining parts of the selected parents' routes. This is done using two operations:

                    **Crossover:** Parts of two parent routes are swapped to create two new offspring routes.

                    **Mutation:** A random change is introduced in a route, for example, swapping the order of two stores.

**Iteration:**

Steps 2-4 are repeated for multiple generations, allowing the population of routes to evolve towards better solutions.

The algorithm stops when a satisfactory solution is found or a maximum number of generations is reached.


##Code Structure

PotentialSolution class: Represents a single route (chromosome) and includes methods for creating random routes, calculating fitness, performing mutations, and crossovers.

AdvancedGuess class: Implements the genetic algorithm, managing the population of routes, selection, reproduction, and stopping criteria.

distanceShuffling class: Inherits from PotentialSolution and is specific to the TSP problem, representing routes between stores, calculating distances, and overriding methods for random route generation, mutation, and crossover.

Main execution block: Creates an initial population, sets up the genetic algorithm, and runs the optimization process.


#Final Interpretatiion

The code uses a genetic algorithm to efficiently explore the vast space of possible routes, gradually improving the solution over generations until a near-optimal route for the salesperson is found. This route minimizes the total travel distance and helps the salesperson efficiently visit all the stores.